In [1]:
%matplotlib inline
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
sys.version_info

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)

In [4]:
from MomentMatching.newMomentMatch import UnscentedTransform, TaylorTransform, MonteCarloTransform
from Systems import UniformNonlinearGrowthModel
from MomentMatching.StateModels import GaussianState
from ExpectationPropagation import EPNodes
from MomentMatching.ExpectationPropagation import TopEP
from Filters.KalmanFilter import KalmanFilterSmoother, PowerKalmanFilterSmoother
from Utils.Metrics import nll, rmse
from Utils.Plot_Helper import plot_gaussian, plot_gaussian_node
from Systems import BearingsOnlyTracking
import logging

In [5]:
SEED = 100

np.random.seed(seed=SEED)

N = 50
system = UniformNonlinearGrowthModel()
# system = BearingsOnlyTracking()
data = system.simulate(N)
x_true, x_noisy, y_true, y_noisy = zip(*data)


power = 1
damping = 1

transform = UnscentedTransform(n=1,  beta=0,  alpha=1, kappa=2)
meas_transform = UnscentedTransform(n=1, beta=0,  alpha=1, kappa=2)

In [6]:
def _power_sweep(power, damping):
    transform = UnscentedTransform(n=1, beta=0, alpha=1, kappa=1)
    meas_transform = UnscentedTransform(n=1, beta=0, alpha=1, kappa=1)
#     transform = TaylorTransform()
#     transform = MonteCarloTransform(number_of_samples=2000)
#     meas_transform = MonteCarloTransform(number_of_samples=2000)
    Nodes = EPNodes(dimension_of_state=1, N=N)
    EP = TopEP(system_model=system,
               moment_matching=transform,
               meas_transform=meas_transform,
               power=power,
               damping=damping)

    EPNodesList = EP.forward_backward_iteration(10, Nodes, y_noisy, list(range(0, N)), x_true)

    Node = [node.marginal for node in EPNodesList[-1]]
    return nll(Node, x_true), rmse(Node, x_true)

In [7]:
x = 10
y = 10
power_range = np.linspace(0.1, 1.0, num=x)
damp_range = np.linspace(0.1, 1.0, num=y)

In [8]:
power_range

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [9]:
results = []
NLL = []
RMSE = []
for power, damping in itertools.product(power_range, damp_range):
    ans = _power_sweep(power, damping)
    a1, a2 = ans
    NLL.append(a1)
    RMSE.append(a2)
    results.append(ans)

In [10]:
results

[(4.0202256626919475, 9.604311309238033),
 (4.099765857161781, 9.642539489207836),
 (4.2240886680996095, 9.651636503477997),
 (4.322341936216337, 9.635572673778),
 (4.349033810180067, 9.684042823855949),
 (4.308703316270749, 9.537507975941615),
 (4.396874237343187, 9.559214960257284),
 (4.534639616357847, 9.421296400044442),
 (4.766660088741683, 9.121444031655367),
 (4.9879944988216645, 9.968450925043753),
 (4.057389554507469, 9.27521140581096),
 (4.074324261259366, 9.569936078077728),
 (4.133340107169323, 10.003829642345208),
 (4.166901219239921, 9.863668907718042),
 (4.256469233642323, 10.177497816815608),
 (4.297297527519077, 9.842604691763425),
 (4.377237491838195, 10.073308936490037),
 (4.42859988146865, 10.568065632525181),
 (4.656339980743074, 10.479752490602143),
 (4.60042026147046, 11.420587263394623),
 (4.098623181351556, 10.644430845763416),
 (4.119051581198399, 9.768355237266553),
 (4.1174889116163085, 10.070576322189197),
 (4.063403292426379, 10.238136821367764),
 (4.04066

In [11]:
len(results)

100

In [12]:
ll = np.array(NLL)
Rmse = np.array(RMSE)

In [13]:
ind_ll = np.array(list(itertools.product(power_range, damp_range)))

In [14]:
ind_ll.reshape(x, y, 2)[0]

array([[0.1, 0.1],
       [0.1, 0.2],
       [0.1, 0.3],
       [0.1, 0.4],
       [0.1, 0.5],
       [0.1, 0.6],
       [0.1, 0.7],
       [0.1, 0.8],
       [0.1, 0.9],
       [0.1, 1. ]])

In [15]:

img_ll = ll.reshape(x, y)
img_rmse = Rmse.reshape(x, y)

In [16]:
np.argmin(img_ll)

68

In [17]:
ind_ll[np.argmin(img_ll)]

array([0.7, 0.9])

In [18]:
ind_ll[np.argmax(img_ll)]

array([1., 1.])

In [19]:
ind_ll[3]

array([0.1, 0.4])

In [20]:
NLL[3]

4.322341936216337

In [21]:
_power_sweep(0.1, 0.4)

(4.322341936216337, 9.635572673778)

In [22]:
img_rmse[6][4]

2.934655356261493

In [23]:
np.max(img_rmse)

16.068254290845793

In [24]:
methods = [None, 'none','nearest', 'bilinear', 'bicubic', 'spline16',
           'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric',
           'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos']

In [25]:
len(methods)

18

In [26]:
# fig, ax = plt.subplots(4,4)

In [27]:
ax[0]

NameError: name 'ax' is not defined

In [ ]:
def plot(img, methods):
    plt.imshow(img_ll, interpolation=methods[9], extent=[0.0,1,0.0,1], cmap='jet', clim=(0, 6))
    ax = plt.gca()
    ax.set_xlabel('Power')
    ax.set_ylabel('Damping')
    ax.grid(False)
    plt.colorbar()

In [ ]:
log_nll = np.log(img_ll)

In [ ]:
plt.imshow(log_nll, interpolation=methods[5], extent=[0.0,1,0.0,1], cmap='jet', vmax=np.max(log_nll), vmin=np.min(log_nll))
ax = plt.gca()
ax.set_xlabel('Power')
ax.set_ylabel('Damping')
ax.grid(False)
plt.colorbar()

In [ ]:
log_rmse = np.log(img_rmse)

In [ ]:
plt.imshow(log_rmse, interpolation=methods[5], extent=[0.1,1,0.1,1], cmap='jet', vmax=np.max(log_rmse), vmin=np.min(log_rmse))
ax = plt.gca()
ax.set_xlabel('Power')
ax.set_ylabel('Damping')
ax.grid(False)
plt.colorbar()

In [ ]:
dx

In [ ]:
x_plot, y_plot = np.mgrid[0.1:1.1:0.1, 0.1:1.1:0.1]

In [ ]:
plt.pcolor( y_plot, x_plot, log_rmse, cmap='RdBu' )
plt.colorbar()
# plt.axis([x_plot.min(), x_plot.max(), y_plot.min(), y_plot.max()])

In [ ]:
for i in range(len(power_range)):
    plt.plot(power_range, img_ll[:, i])

In [ ]:
Rmse